In [3]:
import os
import cv2
import numpy as np
from tqdm import tqdm


REBUILD_DATA = False # set to true to one once, then back to false unless you want to change something in your training data.

class DogsVSCats():
    IMG_SIZE = 50
    CATS = "PetImages/Cat"
    DOGS = "PetImages/Dog"
    TESTING = "PetImages/Testing"
    LABELS = {CATS: 0, DOGS: 1}
    training_data = []

    catcount = 0
    dogcount = 0

    def make_training_data(self):
        for label in self.LABELS:
            print(label)
            for f in tqdm(os.listdir(label)):
                if "jpg" in f:
                    try:
                        path = os.path.join(label, f)
                        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
                        img = cv2.resize(img, (self.IMG_SIZE, self.IMG_SIZE))
                        self.training_data.append([np.array(img), np.eye(2)[self.LABELS[label]]])  # do something like print(np.eye(2)[1]), just makes one_hot 
                        #print(np.eye(2)[self.LABELS[label]])

                        if label == self.CATS:
                            self.catcount += 1
                        elif label == self.DOGS:
                            self.dogcount += 1

                    except Exception as e:
                        pass
                        #print(label, f, str(e))

        np.random.shuffle(self.training_data)
        np.save("training_data.npy", self.training_data)
        print('Cats:',dogsvcats.catcount)
        print('Dogs:',dogsvcats.dogcount)

if REBUILD_DATA:
    dogsvcats = DogsVSCats()
    dogsvcats.make_training_data()


training_data = np.load("training_data.npy", allow_pickle=True)
print(len(training_data))

  0%|▏                                                                             | 40/12501 [00:00<00:31, 396.36it/s]

PetImages/Cat


  0%|▎                                                                             | 48/12501 [00:00<00:26, 475.38it/s]

PetImages/Dog


100%|███████████████████████████████████████████████████████████████████████████| 12501/12501 [00:28<00:00, 438.24it/s]


Cats: 12476
Dogs: 12470
24946


In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [25]:
class Net(nn.Module):
    def __init__(self):
        super().__init__() # to run the init of the parent class (nn.Module)
        self.conv1 = nn.Conv2d(1,32,5) # input is 1 image, 32 output channels, 5x5 kernel / window
        self.conv2 = nn.Conv2d(32,64,5) # input is 32, bc the first layer output 32. Then we say the output will be 64 channels, 5x5 conv
        self.conv3 = nn.Conv2d(64,128,5)
        
        x = torch.randn(50,50).view(-1,1,50,50)
        self._to_linear = None
        # This is 50x50 since that is the shape of our image
        # view(-1 = however many feature sets we want, then 1 50x50 image)
        # note this will be the shape of the tensor we actually pass through our neural network
        self.convs(x)
        
        self.fc1 = nn.Linear(self._to_linear,512) #self.to_linear has its number determined by the forward pass
        self.fc2 = nn.Linear(512,1)
        
    def convs(self,x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv3(x)), (2,2))
        
        print(x[0].shape)
        if self._to_linear is None:
             self._to_linear = x[0].shape[0]*x[0].shape[1]*x[0].shape[2]
        return x
    
    def forward(self,x):
        x = self.convs(x) # pass through all convolutional layers
        x = x.view(-1,self._to_linear) # flatten it
        x = F.relu(self.fc1(x)) #  pass through first fully connected layer 
        x = self.fc2(x)
        return F.softmax(x, dim=1)
net = Net()

torch.Size([128, 2, 2])


In [26]:
import torch.optim as optim

optimizer = optim.Adam(net.parameters(), lr=0.001)
loss_function = nn.MSELoss() 

X = torch.Tensor([i[0] for i in training_data]).view(-1,50,50)
X = X/255 #scale imagery
y = torch.Tensor([i[1] for i in training_data])

VAL_PCT = 0.1 #out of sample percent
val_size = int(len(X)*VAL_PCT)
print(len(X))
print(val_size)


24946
2494


In [33]:
train_X = X[:-val_size]
train_y = y[:-val_size]

test_X = X[-val_size:]
test_y = y[-val_size:]

print(len(train_X))
print(len(test_X))

22452
2494


In [34]:
BATCH_SIZE = 100 # make smaller if memory error occurs

EPOCHS = 1

for epoch in range(EPOCHS):
    for i in tqdm(range(0, len(train_X), BATCH_SIZE)):
        print(i, i+BATCH_SIZE)
        batch_X = train_X[i:i+BATCH_SIZE].view(-1,1,50,50)
        batch_y = train_y[i:i+BATCH_SIZE]
        
        net.zero_grad()
        outputs = net(batch_X)
        loss = loss_function(outputs, batch_y)
        loss.backward()
        optimizer.step()
print(loss)

  0%|                                                                                          | 0/225 [00:00<?, ?it/s]

0 100
torch.Size([128, 2, 2])


  0%|▎                                                                                 | 1/225 [00:00<01:08,  3.29it/s]

100 200
torch.Size([128, 2, 2])


  1%|▋                                                                                 | 2/225 [00:00<01:06,  3.37it/s]

...

22200 22300
torch.Size([128, 2, 2])


 99%|███████████████████████████████████████████████████████████████████████████████▎| 223/225 [00:56<00:00,  3.97it/s]

22300 22400
torch.Size([128, 2, 2])


100%|████████████████████████████████████████████████████████████████████████████████| 225/225 [00:57<00:00,  3.95it/s]

22400 22500
torch.Size([128, 2, 2])
tensor(0.5000, grad_fn=<MseLossBackward>)


In [39]:
correct = 0
total = 0

with torch.no_grad():
    for i in tqdm(range(len(test_X))):
        real_class = torch.argmax(test_y[i])
        net_out = net(test_X[i].view(-1,50,50))[0]
        print(real_class, net_out)
        predicted_class = torch.argmax(net_out)
        if predicted_class == real_class:
            correct += 1
        total +=1
        
print("Accuracy:", round(correct/total,3))

  0%|                                                                                         | 0/2494 [00:00<?, ?it/s]


RuntimeError: Expected 4-dimensional input for 4-dimensional weight [32, 1, 5, 5], but got 3-dimensional input of size [1, 50, 50] instead